In [1]:
import sys  
reload(sys)  
sys.setdefaultencoding('utf-8')

In [2]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

In [3]:
import numpy as np
import tldextract
from collections import Counter

In [4]:
import pandas as pd
import json
import timeit
import cPickle as pickle

In [5]:
import os

In [6]:
os.chdir("/Users/nhu2000/")

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils # For y values


In [8]:
def my_pickle_dump(fname, data):
    with open(fname, 'w') as f:
        pickle.dump(data, f)

def my_pickle_load(fname):
    with open(fname) as f:
        return pickle.load(f)

In [9]:
#create a model
model = Sequential()

In [10]:
with open('/Users/nhu2000/desktop/huizhou/data/stop_words/Baidu_Stop_Words.json') as data_file:    
    data = json.load(data_file)

In [11]:
bd_stop_words = list(data)

In [12]:
vectorizer = TfidfVectorizer(analyzer = 'word', stop_words = bd_stop_words)

In [18]:
df_no_dup = pd.read_csv('/Users/nhu2000/desktop/huizhou/data/huiwenhua/huizhou_web_contentv5.csv')

In [19]:
df_no_dup.info()

In [20]:
df_no_dup.shape

(8768, 9)

In [21]:
df_target = df_no_dup[['target']]

In [22]:
# mask for train and test
msk = np.random.rand(len(df_no_dup)) < 0.8
X_train = df_no_dup[msk]
X_test = df_no_dup[~msk]
Y_train = df_target[msk]
Y_test = df_target[~msk]


In [23]:
print X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

In [24]:
train_articles = X_train['segment'].values
test_articles = X_test['segment'].values

In [25]:
X_train_vec = vectorizer.fit_transform(train_articles)
X_test_vec = vectorizer.transform(test_articles)

In [26]:
print X_train_vec.shape, X_test_vec.shape

In [27]:
dimof_input = X_train_vec.shape[1]
dimof_output = 2

In [28]:
print dimof_input, dimof_output

In [29]:
X = X_train_vec.todense()

In [30]:
print type(X), X.shape

In [31]:
Y = np_utils.to_categorical(Y_train.values, dimof_output)

In [32]:
print type(Y), Y.shape

In [33]:
model = Sequential()

In [34]:
batch_size = 128
dimof_middle = 1000 #100
dropout = 0.2
countof_epoch = 100
verbose = 0

In [35]:
model.add(Dense(dimof_middle, input_dim=dimof_input, init='uniform', activation='tanh'))
model.add(Dropout(dropout))
model.add(Dense(dimof_middle, init='uniform', activation='tanh'))
model.add(Dropout(dropout))
model.add(Dense(dimof_output, init='uniform', activation='softmax'))
#model.compile(loss='mse', optimizer='sgd', metrics=['accuracy'])

In [36]:
from keras.optimizers import SGD
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True))

In [37]:
start_time = timeit.default_timer()
model.fit(
    X, Y,
validation_split=0.2,
batch_size=batch_size, nb_epoch=countof_epoch, verbose=verbose)
elapsed = timeit.default_timer() - start_time

In [38]:
print start_time, elapsed

In [39]:
X = X_test_vec.todense()

In [40]:
Y_pred = model.predict_classes(X, batch_size=128)

In [41]:
scores = metrics.accuracy_score(Y_test.values, Y_pred)

In [43]:
print scores  #0.79334/0.9675(optimum)

In [39]:
#Dump model to file
model.save_weights("/Users/nhu2000/desktop/huizhou/keras/keras_opt_model.h5")
json_data = model.to_json()
with open('/Users/nhu2000/desktop/huizhou/keras/keras_opt_model.json', 'w') as outfile:
    json.dump(json_data, outfile)

In [40]:
from keras.models import model_from_json
with open('/Users/nhu2000/desktop/huizhou/keras/keras_opt_model.json') as data_file:    
    json_data = json.load(data_file)

model = model_from_json(json_data)
model.load_weights("/Users/nhu2000/desktop/huizhou/keras/keras_opt_model.h5")